In [1]:
%pip install folium -q

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np

import folium
from folium.plugins import HeatMap, MousePosition
from folium import plugins

pd.set_option('display.max_columns', None)

# Exercício 2

No anexo deste e-mail você encontra um arquivo intitulado “INSUMOS PARA ETAPA 2 - EXERCÍCIO 2” que corresponde a um extrato da base de dados antes do processo de limpeza. Considerando as informações do dicionário de dados, analise a base e identifique os IDs que possuem erros a serem corrigidos, identificando quais os erros você identificou.

## Verificações feitas

- ~~IDs duplicados~~
- ~~Verificar unidades policiais em casos sem agentes~~
- ~~Datas incorretas~~
- ~~Verificar Lat e Lon incorretos~~
- ~~presenca_agentes e número de agentes feridos ou mortos~~
- ~~Motivo princial inválido~~
- ~~Ocorrência com 3 ou mais mortos que não foram categorizados como chacina~~
- ~~Ação policial sem o registro de presença de agentes~~
- ~~Ocorrência de ação policial sem agentes presentes~~

## IDs Irregulares 

### Ocorrências de UF que não sejam do RJ, PE ou BA
- 169 regitros do Pará

### Ocorrências com bairro em branco
-  67951, 67902, 66994

### Ocorrência com registro de agentes mortos mas registro de presença de agentes consta como Não

- 68166, 68139, 67892, 67375, 66888

### Registros alfanuméricos inconsistentes na coluna ocorrencia_relacionada (numérica):  

- 68140, 68107, 67900, 67544, 67540, 67508, 66732

In [3]:
df = pd.read_excel('../data/dados_ex2.xlsx')

In [4]:
df.shape

(1366, 24)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1366 entries, 0 to 1365
Data columns (total 24 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   id_ocorrencia               1366 non-null   int64         
 1   status                      1366 non-null   object        
 2   uf                          1366 non-null   object        
 3   cidade                      1366 non-null   object        
 4   endereco                    1366 non-null   object        
 5   bairro                      1363 non-null   object        
 6   sub_bairro                  103 non-null    object        
 7   localidade                  423 non-null    object        
 8   latitude                    1366 non-null   float64       
 9   longitude                   1366 non-null   float64       
 10  data_ocorrencia             1366 non-null   datetime64[ns]
 11  presenca_agentes            1366 non-null   object      

In [6]:
df.describe().T

,count,mean,min,25%,50%,75%,max,std
id_ocorrencia,1366.0,67447.268668,66722.0,67083.25,67444.5,67811.75,68177.0,421.600082
latitude,1366.0,-13.910111,-23.030586,-22.796499,-12.929894,-8.063028,-1.141922,7.488636
longitude,1366.0,-40.609059,-48.701344,-43.35466,-38.514111,-38.305466,-34.811849,4.336357
data_ocorrencia,1366,2024-10-10 04:41:51.874084864,2024-06-01 10:00:00,2024-09-21 22:00:00,2024-10-11 15:42:30,2024-10-29 20:00:00,2024-11-18 23:33:00,NaN
civis_mortos_ocorrencia,750.0,1.154667,1.0,1.0,1.0,1.0,5.0,0.495721
civis_feridos_ocorrencia,272.0,1.319853,1.0,1.0,1.0,1.0,6.0,0.706253
agentes_mortos_ocorrencia,19.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0
agentes_feridos_ocorrencia,21.0,1.238095,1.0,1.0,1.0,1.0,3.0,0.538958


In [7]:
df.describe(include='object').T

,count,unique,top,freq
status,1366,1,Approved,1366
uf,1366,4,Rio de Janeiro,487
cidade,1366,51,SALVADOR,293
endereco,1366,1286,"Morro dos Macacos - Vila Isabel, Rio de Janeir...",6
bairro,1363,558,NI,35
sub_bairro,103,64,NOSSA SENHORA DA LUZ,7
localidade,423,284,MORRO DOS MACACOS,12
presenca_agentes,1366,2,Não,863
acao_policial,1366,2,Não,924
chacina_ocorrencia,1366,2,Não,1341


In [8]:
df['motivo_principal'].unique()

array(['Ação policial', 'Não identificado', 'Homicidio/Tentativa',
       'Suicídio', 'Tentativa/Roubo', 'Operação policial', 'Disputa',
       'Tortura', 'Briga', 'Tiros a esmo', 'Tentativa/Roubo de cargas',
       'Ataque a civis', 'Outro', 'Sequestro/Cárcere Privado',
       'Disparo Acidental'], dtype=object)

In [9]:
df.head()

,id_ocorrencia,status,uf,cidade,endereco,bairro,sub_bairro,localidade,latitude,longitude,data_ocorrencia,presenca_agentes,acao_policial,civis_mortos_ocorrencia,civis_feridos_ocorrencia,agentes_mortos_ocorrencia,agentes_feridos_ocorrencia,chacina_ocorrencia,unidades_policiais,motivo_principal,motivos_complementares,outros_recortes,transporte_interrompido,ocorrencia_relacionada
0,68177,Approved,Rio de Janeiro,RIO DE JANEIRO,"Catiri - Bangu, Rio de Janeiro - RJ, 21863-010...",BANGU,NaN,CATIRI,-22.853198,-43.470591,2024-11-17 10:00:00,Sim,Sim,NaN,1.0,NaN,NaN,Não,14 BPM,Ação policial,NaN,NaN,Não,NaN
1,68176,Approved,Rio de Janeiro,RIO DE JANEIRO,"Cordovil, Rio de Janeiro - RJ, Brasil",CORDOVIL,NaN,NaN,-22.823901,-43.301091,2024-11-18 23:33:00,Não,Não,NaN,NaN,NaN,NaN,Não,NaN,Não identificado,NaN,NaN,Não,NaN
2,68175,Approved,Rio de Janeiro,JAPERI,"Eng. Pedreira, Japeri - RJ, 26460-340, Brasil",CENTRO DE ENGENHEIRO PEDREIRA,NaN,NaN,-22.677115,-43.607651,2024-11-18 20:00:00,Sim,Sim,NaN,2.0,NaN,NaN,Não,24 BPM,Ação policial,NaN,NaN,Não,NaN
3,68174,Approved,Rio de Janeiro,JAPERI,"Jardim Aljezur, Japeri - RJ, Brasil",ALJEZUR,NaN,NaN,-22.679292,-43.578714,2024-11-18 08:00:00,Sim,Sim,2.0,NaN,NaN,NaN,Não,NaN,Homicidio/Tentativa,NaN,Outros,Não,NaN
4,68173,Approved,Bahia,SALVADOR,"Av. Vasco da Gama, Engenho Velho de Brotas, S...",ENGENHO VELHO DE BROTAS,NaN,NaN,-12.992244,-38.501418,2024-11-18 19:00:00,Não,Não,NaN,1.0,NaN,NaN,Não,NaN,Homicidio/Tentativa,Não identificado,NaN,Não,NaN


In [10]:
df['uf'].unique()

array(['Rio de Janeiro', 'Bahia', 'Pará', 'Pernambuco'], dtype=object)

In [11]:
df['cidade'].unique()

array(['RIO DE JANEIRO', 'JAPERI', 'SALVADOR', 'ANANINDEUA', 'BELEM',
       'CASTANHAL', 'CABO DE SANTO AGOSTINHO', 'RECIFE', 'BENEVIDES',
       'MARITUBA', 'JABOATAO DOS GUARARAPES', 'NITERÓI', 'CAMACARI',
       "DIAS D'AVILA", 'BELFORD ROXO', 'MATA DE SAO JOAO',
       'SÃO JOÃO DE MERITI', 'IGARASSU', 'SANTA IZABEL DO PARA',
       'SAO LOURENCO DA MATA', 'GOIANA', 'LAURO DE FREITAS',
       'DUQUE DE CAXIAS', 'CANDEIAS', 'MORENO', 'CAMARAGIBE', 'PAULISTA',
       'SIMOES FILHO', 'SÃO GONÇALO', 'NOVA IGUAÇU', 'OLINDA', 'ITABORAÍ',
       'SANTA BARBARA DO PARA', 'VERA CRUZ', 'BARCARENA', 'ABREU E LIMA',
       'MESQUITA', 'TANGUÁ', 'MARICÁ', 'ILHA DE ITAMARACA', 'RIO BONITO',
       'ITAPARICA', 'IPOJUCA', 'MADRE DE DEUS', 'SEROPÉDICA',
       'ITAPISSUMA', 'MAGÉ', 'QUEIMADOS', 'NILÓPOLIS', 'ARACOIABA',
       'CACHOEIRAS DE MACACU'], dtype=object)

In [12]:
df['bairro'].unique()

array(['BANGU', 'CORDOVIL', 'CENTRO DE ENGENHEIRO PEDREIRA', 'ALJEZUR',
       'ENGENHO VELHO DE BROTAS', 'DISTRITO INDUSTRIAL', 'TENONE',
       'CAICARA', 'GAIBU', 'TAQUARA', 'IMBIRIBEIRA', 'JARDIM SAO PAULO',
       'NI', 'CANAA', 'ALMIR GABRIEL', 'SUCUPIRA', 'CURADO', 'AFLITOS',
       'TORROES', 'COQUEIRAL', 'JACAREPAGUA', 'FONSECA', 'SAO CRISTOVAO',
       'ITACARANHA', 'CIDADE DE DEUS', 'JARDIM LIMOEIRO', 'LAMA PRETA',
       'SAO TOME', 'LINS DE VASCONCELOS', 'BRAS DE PINA',
       'FREGUESIA (JACAREPAGUA)', 'VERDES HORIZONTES',
       'MORADA DAS MANGUEIRAS', 'CAMPO GRANDE', 'CANUDOS', 'PERNAMBUES',
       'IAPI', 'PARQUE GUAJARA', 'CENTRO', 'RIO SENA', 'ACUZINHO',
       'VILAR DOS TELES', 'BAIRRO DA PAZ', 'AGAMENON', 'CAMPINHO', 'EDEN',
       'BARRO', 'NOVO BAIRRO', 'XAVANTES', 'JARDIM MIRAI', 'CIDADE NOVA',
       'AGULHA', 'MURIBARA', 'PONTE DOS CARVALHOS', 'GOIANA',
       'CAJUEIRO SECO', 'CAJI', 'FREGUESIA (ILHA)', 'MARACACUERA',
       'CENTENARIO', 'GARDENIA AZUL', '

In [13]:
df['data_ocorrencia']

0      2024-11-17 10:00:00
1      2024-11-18 23:33:00
2      2024-11-18 20:00:00
3      2024-11-18 08:00:00
4      2024-11-18 19:00:00
               ...        
1361   2024-09-01 16:30:00
1362   2024-09-02 11:30:00
1363   2024-09-02 00:20:00
1364   2024-09-01 00:00:00
1365   2024-09-01 22:00:00
Name: data_ocorrencia, Length: 1366, dtype: datetime64[ns]

In [14]:
df['ocorrencia_relacionada'].unique()

array([nan, '66520\n67357\n67961', '68046\n', '68046', '67967', '67970',
       '#67899', '67485', ' ', '67509\n\n', '#67500', '67159', '#66731'],
      dtype=object)

In [15]:
df.query('civis_mortos_ocorrencia >= 3')['chacina_ocorrencia'].unique()

array(['Sim'], dtype=object)

In [16]:
df.query('acao_policial == "Sim"')['presenca_agentes'].unique()

array(['Sim'], dtype=object)

In [17]:
df['agentes_mortos_ocorrencia'].unique()

array([nan,  1.])

In [18]:
limites_br = 'geojs-100-mun.json'

In [19]:
coords = df.groupby(['latitude', 'longitude']).count()['id_ocorrencia'].reset_index().values.tolist()

In [20]:
base_map = folium.Map(
    width='100%',
    height='100%',
    location = [-22.27, -42.47],
    zoom_start=9
)


# Adicionando o mapa de calor de acidentes por área
HeatMap(coords).add_to(base_map)

In [21]:
base_map

In [22]:
df.query('presenca_agentes == "Não" and agentes_mortos_ocorrencia >= 1')[[
    'id_ocorrencia', 
    'uf', 
    'data_ocorrencia', 
    'presenca_agentes',
    'agentes_mortos_ocorrencia', 
    'agentes_feridos_ocorrencia'
]]

,id_ocorrencia,uf,data_ocorrencia,presenca_agentes,agentes_mortos_ocorrencia,agentes_feridos_ocorrencia
10,68166,Pernambuco,2024-11-17 00:00:00,Não,1.0,NaN
37,68139,Bahia,2024-11-17 03:00:00,Não,1.0,NaN
268,67892,Pará,2024-11-03 02:00:00,Não,1.0,NaN
748,67375,Pará,2024-10-08 15:30:00,Não,1.0,NaN
1207,66888,Pará,2024-09-10 18:30:00,Não,1.0,NaN


In [23]:
df.query('presenca_agentes == "Sim"')['acao_policial'].value_counts()

acao_policial
Sim    442
Não     61
Name: count, dtype: int64

In [24]:
sum(df['ocorrencia_relacionada'].notnull())

12

In [25]:
df.loc[df['ocorrencia_relacionada'].notnull()][['id_ocorrencia', 'ocorrencia_relacionada']]

,id_ocorrencia,ocorrencia_relacionada
36,68140,66520\n67357\n67961
68,68107,68046\n
115,68055,68046
185,67980,67967
190,67975,67970
261,67900,#67899
538,67600,67485
590,67544,
594,67540,67509\n\n
623,67508,#67500


In [26]:
df.query('presenca_agentes == "Não"')['unidades_policiais'].value_counts()

unidades_policiais
NÃO IDENTIFICADO     1
Name: count, dtype: int64

In [27]:
df['motivo_principal'].value_counts()

motivo_principal
Homicidio/Tentativa          562
Ação policial                328
Não identificado             155
Tentativa/Roubo              126
Disputa                       78
Operação policial             59
Briga                         19
Tiros a esmo                  12
Ataque a civis                 8
Tentativa/Roubo de cargas      6
Tortura                        4
Outro                          4
Suicídio                       2
Disparo Acidental              2
Sequestro/Cárcere Privado      1
Name: count, dtype: int64

In [28]:
df.query('motivo_principal == "Ação policial"')

,id_ocorrencia,status,uf,cidade,endereco,bairro,sub_bairro,localidade,latitude,longitude,data_ocorrencia,presenca_agentes,acao_policial,civis_mortos_ocorrencia,civis_feridos_ocorrencia,agentes_mortos_ocorrencia,agentes_feridos_ocorrencia,chacina_ocorrencia,unidades_policiais,motivo_principal,motivos_complementares,outros_recortes,transporte_interrompido,ocorrencia_relacionada
0,68177,Approved,Rio de Janeiro,RIO DE JANEIRO,"Catiri - Bangu, Rio de Janeiro - RJ, 21863-010...",BANGU,NaN,CATIRI,-22.853198,-43.470591,2024-11-17 10:00:00,Sim,Sim,NaN,1.0,NaN,NaN,Não,14 BPM,Ação policial,NaN,NaN,Não,NaN
2,68175,Approved,Rio de Janeiro,JAPERI,"Eng. Pedreira, Japeri - RJ, 26460-340, Brasil",CENTRO DE ENGENHEIRO PEDREIRA,NaN,NaN,-22.677115,-43.607651,2024-11-18 20:00:00,Sim,Sim,NaN,2.0,NaN,NaN,Não,24 BPM,Ação policial,NaN,NaN,Não,NaN
6,68171,Approved,Pará,BELEM,"Tenoné, Belém - PA, Brasil",TENONE,NaN,NaN,-1.318947,-48.441036,2024-11-18 14:00:00,Sim,Sim,1.0,NaN,NaN,NaN,Não,ROTAM,Ação policial,NaN,NaN,Não,NaN
7,68170,Approved,Pará,CASTANHAL,Tv. Benjamin Constant (Feira da Ceasa) - Caica...,CAICARA,NaN,NaN,-1.288548,-47.923166,2024-11-16 19:00:00,Sim,Sim,NaN,1.0,NaN,NaN,Não,NÃO IDENTIFICADO,Ação policial,NaN,NaN,Não,NaN
8,68168,Approved,Pernambuco,CABO DE SANTO AGOSTINHO,"Comunidade da Mangueirinha, Cabo de Santo Agos...",GAIBU,NaN,COMUNIDADE DA MANGUEIRINHA,-8.285247,-35.062217,2024-11-16 00:00:00,Sim,Sim,NaN,1.0,NaN,NaN,Não,Polícia Militar,Ação policial,Briga,NaN,Não,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1318,66771,Approved,Pará,BELEM,"Al. Elna Trindade - Mangueiras, Belém - PA, 66...",MANGUEIRAS,NaN,NaN,-1.147245,-48.453055,2024-09-03 22:20:00,Sim,Sim,1.0,NaN,NaN,NaN,Não,25°BPM,Ação policial,NaN,NaN,Não,NaN
1341,66748,Approved,Rio de Janeiro,RIO DE JANEIRO,"Vila do João (Complexo da Maré) - Maré, Rio de...",MARE,NaN,VILA DO JOAO (COMPLEXO DA MARE),-22.871176,-43.241085,2024-09-03 12:28:00,Sim,Sim,NaN,NaN,NaN,NaN,Não,"BOPE, CORE, DRE",Ação policial,NaN,NaN,Não,NaN
1355,66733,Approved,Bahia,SALVADOR,"Baixa da Jega, Bairro 17 , Itapuã, Salvador - BA",ITAPUA,BAIRRO 17 (KM 17),BAIXA DA JEGA (KM 17 - ITAPUA),-12.932293,-38.364056,2024-09-02 20:00:00,Sim,Sim,1.0,NaN,NaN,NaN,Não,RONDESP ATLÂNTICO,Ação policial,Não identificado,NaN,Não,NaN
1358,66730,Approved,Pará,BELEM,"R. da Pedreira - Aeroporto, Belém - PA, 66915-...",AEROPORTO,NaN,NaN,-1.141922,-48.455780,2024-09-01 21:30:00,Sim,Sim,1.0,NaN,NaN,NaN,Não,25°BPM,Ação policial,NaN,NaN,Não,NaN


In [29]:
df['outros_recortes'].value_counts()

outros_recortes
Perseguição                                         45
Ataques armados sobre rodas                         33
Bar                                                 18
Presença de político                                10
Feminicídio/tentativa                                7
Outros                                               5
Perseguição,Perseguição                              1
Quilombo                                             1
Ataques armados sobre rodas,Bar                      1
Presença de político,Ataques armados sobre rodas     1
Perseguição,Ataques armados sobre rodas              1
Ataques armados sobre rodas,Presença de político     1
Ataques armados sobre rodas,Outros                   1
Bar,Ataques armados sobre rodas                      1
Name: count, dtype: int64

In [30]:
df.query('outros_recortes == "Perseguição,Perseguição"')

,id_ocorrencia,status,uf,cidade,endereco,bairro,sub_bairro,localidade,latitude,longitude,data_ocorrencia,presenca_agentes,acao_policial,civis_mortos_ocorrencia,civis_feridos_ocorrencia,agentes_mortos_ocorrencia,agentes_feridos_ocorrencia,chacina_ocorrencia,unidades_policiais,motivo_principal,motivos_complementares,outros_recortes,transporte_interrompido,ocorrencia_relacionada
15,68161,Approved,Pará,MARITUBA,"Almir Gabriel, Marituba - PA, 67200-000, Brasil",ALMIR GABRIEL,NaN,NaN,-1.373889,-48.306263,2024-11-17 21:20:00,Sim,Sim,2.0,NaN,NaN,NaN,Não,21° BPM,Tentativa/Roubo,"Sequestro/Cárcere Privado,Operação policial","Perseguição,Perseguição",Não,NaN


In [31]:
df.query('chacina_ocorrencia == "Sim"')['_mortos_ocorrencia'].value_counts()

KeyError: '_mortos_ocorrencia'

In [ ]:
df.query('motivo_principal == "Ataque a civis"')[['civis_feridos_ocorrencia', 'civis_mortos_ocorrencia']]

,civis_feridos_ocorrencia,civis_mortos_ocorrencia
355,2.0,NaN
559,1.0,2.0
778,NaN,NaN
818,1.0,NaN
827,1.0,3.0
1137,2.0,NaN
1249,NaN,NaN
1264,2.0,NaN


In [ ]:
df.loc[df['motivo_principal'].isnull()]

,id_ocorrencia,status,uf,cidade,endereco,bairro,sub_bairro,localidade,latitude,longitude,data_ocorrencia,presenca_agentes,acao_policial,civis_mortos_ocorrencia,civis_feridos_ocorrencia,agentes_mortos_ocorrencia,agentes_feridos_ocorrencia,chacina_ocorrencia,unidades_policiais,motivo_principal,motivos_complementares,outros_recortes,transporte_interrompido,ocorrencia_relacionada


In [ ]:
df['uf'].value_counts()

uf
Rio de Janeiro    487
Bahia             399
Pernambuco        311
Pará              169
Name: count, dtype: int64

In [ ]:
df.loc[df['uf'] == 'Pará']

,id_ocorrencia,status,uf,cidade,endereco,bairro,sub_bairro,localidade,latitude,longitude,data_ocorrencia,presenca_agentes,acao_policial,civis_mortos_ocorrencia,civis_feridos_ocorrencia,agentes_mortos_ocorrencia,agentes_feridos_ocorrencia,chacina_ocorrencia,unidades_policiais,motivo_principal,motivos_complementares,outros_recortes,transporte_interrompido,ocorrencia_relacionada
5,68172,Approved,Pará,ANANINDEUA,Tv. Antonio Moutinho (Residencial Clodomir Naz...,DISTRITO INDUSTRIAL,NaN,NaN,-1.358361,-48.361143,2024-11-18 16:00:00,Não,Não,NaN,NaN,NaN,NaN,Não,NÃO IDENTIFICADO,Não identificado,NaN,NaN,Não,NaN
6,68171,Approved,Pará,BELEM,"Tenoné, Belém - PA, Brasil",TENONE,NaN,NaN,-1.318947,-48.441036,2024-11-18 14:00:00,Sim,Sim,1.0,NaN,NaN,NaN,Não,ROTAM,Ação policial,NaN,NaN,Não,NaN
7,68170,Approved,Pará,CASTANHAL,Tv. Benjamin Constant (Feira da Ceasa) - Caica...,CAICARA,NaN,NaN,-1.288548,-47.923166,2024-11-16 19:00:00,Sim,Sim,NaN,1.0,NaN,NaN,Não,NÃO IDENTIFICADO,Ação policial,NaN,NaN,Não,NaN
12,68164,Approved,Pará,BELEM,"Conj. Celso Daniel - Tenoné, Belém - PA, 66820...",TENONE,NaN,NaN,-1.322983,-48.439284,2024-11-17 22:30:00,Não,Não,NaN,1.0,NaN,NaN,Não,NaN,Tentativa/Roubo,NaN,NaN,Não,NaN
13,68163,Approved,Pará,BENEVIDES,"Benevides, PA, 68795-000, Brasil",NI,NaN,NaN,-1.358682,-48.249989,2024-11-15 13:00:00,Sim,Sim,1.0,NaN,NaN,NaN,Não,Não identificado,Ação policial,NaN,NaN,Não,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1335,66754,Approved,Pará,MARITUBA,Alça Viária do Pará - BR-155 (PA-483) - Maritu...,SANTA LUCIA,NaN,NaN,-1.402515,-48.339449,2024-09-01 11:00:00,Sim,Sim,NaN,1.0,NaN,NaN,Não,Batalhão de Polícia Rodoviária (BPRv),Operação policial,NaN,NaN,Não,NaN
1347,66742,Approved,Pará,CASTANHAL,"Castanhal, PA, Brasil",NI,NaN,NaN,-1.298267,-47.906946,2024-09-03 08:00:00,Sim,Sim,1.0,NaN,NaN,NaN,Não,PC,Operação policial,Não identificado,NaN,Não,NaN
1358,66730,Approved,Pará,BELEM,"R. da Pedreira - Aeroporto, Belém - PA, 66915-...",AEROPORTO,NaN,NaN,-1.141922,-48.455780,2024-09-01 21:30:00,Sim,Sim,1.0,NaN,NaN,NaN,Não,25°BPM,Ação policial,NaN,NaN,Não,NaN
1359,66729,Approved,Pará,BELEM,"Al. São Francisco - Marco, Belém - PA, 66070-2...",MARCO,NaN,NaN,-1.447399,-48.455520,2024-09-02 18:30:00,Não,Não,1.0,NaN,NaN,NaN,Não,NaN,Homicidio/Tentativa,NaN,Ataques armados sobre rodas,Não,NaN


In [34]:
df['motivo_principal'].unique()

array(['Ação policial', 'Não identificado', 'Homicidio/Tentativa',
       'Suicídio', 'Tentativa/Roubo', 'Operação policial', 'Disputa',
       'Tortura', 'Briga', 'Tiros a esmo', 'Tentativa/Roubo de cargas',
       'Ataque a civis', 'Outro', 'Sequestro/Cárcere Privado',
       'Disparo Acidental'], dtype=object)

In [40]:
df.query('motivo_principal == "Ação policial"')['presenca_agentes'].value_counts()

presenca_agentes
Sim    328
Name: count, dtype: int64